# Tutorial 2 - Page rank, plotting and more numpy gymnastics

In this exercise, we will implement and play with the page rank algorithm.
As seen in the lecture, the page rank algorithm allows to take a graph (e.g. the internet graph) and rank its nodes (the websites) in order of importance, or relevance, to the rest of the graph.  Of course, we can apply the page rank algorithm also to other graphs, not only to the internet.

In exercise 1, we play with the novel "Les Miserables". We construct the graph of co-occurencies of the characters: each character of the novel is a node of the graph (a website in the internet example) and each pair of characters is linked if they appear together in at least one scene. We will use pagerank to identify the most relevant characters of the novel.
We will then compare the pagerank algorithm with the direct computation of the leading eigenvector using numpy's eigen-decomposition utilities.

In exercise 2, we implement page rank for a much larger graph, the graph of links in the wikipedia of simple english. This will require some care in the implementation to run efficiently - for this we will use the sparse matrix format and think about how to re-express a single pagerank iteration more efficiently.

Exercise 3 consists in a few theoretical questions similar to those asked in the exam.

The theoretical tools used in this tutorial are those of section 1 of the lecture notes (eigenvalues, the power method and the page rank algorithm).

# Exercise 1 : Les Miserables

In [ ]:
# importing libraries we need later
import numpy as np
import matplotlib.pyplot as plt

## 1.a Loading the data and visualization
For starters, we go for the dataset of co-occurences in a scene between the characters of "Les Miserables". This containes 77 characters and counts for every pairs if they appear together. 

Note that in the beginning of this notebook, we already imported all the proprietary libraries we will need. You know `numpy` for arrays and vectors, but we added `matplotlib` we will use for plotting later. 

To load the data from Les Miserables, the TAs wrote some code to import it. Since it is not important that you read/understand it, we put the code in a separate file in this directory `utils.py`.

To use the functions defined in there for data loading and visualization, we simply use:

In [ ]:
# import from current directory, a function to visualize the toy graph
from utils import load_les_miserables, plot_les_miserables

It is a nice way to keep code orderly and reuse it later. For more info on this, see [this Tutorial](https://www.geeksforgeeks.org/python-modules/).

You can now use these functions like any other function coming from a library like numpy. You do not need to use a prefix like `np.your_function()` though, because we imported the functions directly via the `from ... import ...` statement.

In [ ]:
# This function returns
# n: number of characters
# names: of the characters
# A: (n,n) matrix where A[i,j] counts common appearences
n, names, A = load_les_miserables()

- Given the indices associated to character 0 and character 17, find their names and, check whether these two co-occur in the story. Compute the number of appearances each one of them has. Feel free to use [f-strings](https://realpython.com/python-f-strings/)

In [ ]:
# TO FILL

In [ ]:
# TO FILL

In [ ]:
# TO FILL

To get a feeling for how the matrix looks like, we want to plot it using `plt.imshow` from the [matplotlib library](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html), probably the most basic and important library for plotting in python.
By sending commands to the `plt` module, one procedurally builds a plot.
- Use `plt.imshow` to display the matrix $A$ and title the plot 'The adjacency matrix A'.

In [ ]:
# TO FILL

- What property can you deduce about the network from the image ? Does this make sense given the considered data ?

In [ ]:
# TO FILL

## 1.b Creating the page rank matrix $G$

We now want to find the PageRank $w_i$ of every member $i$ of the Les Miserables cast. How can we interpret such a value? 
Essentially, $w_i$ will be high for members that have many appearences with different cast members, i.e. that are 'central' in the cast graph.

To find out who these members are, let's go on to define the matrix used in the algorithm using the notation from the lecture:

Recall from the lecture, the definition of the pagerank matrix $G$.
$$
G = (1-\epsilon)S+\epsilon \frac{1}{n}\\
$$
with
$$
S_{ij} = A_{ij}/d_j\quad \mathrm{if } \,\,\,d_j > 0\ ,\quad
S_{ij} = 1/n\quad \mathrm{if }\,\,\,d_j = 0
$$
and $A$ is the adjacency matrix of the underlying graph, so $A_{ij}=1$ if there is an link from $j$ to $i$ and it is zero otherwise.
Since every node in our current example is connected, we can brush over the case $d_j=0$ for now.

In [ ]:
# set the dampening parameter
eps = 0.05

Compute the outgoing edges that a node $i$ has by summing over the column $d_j = \sum_i A_{ij}$

In [ ]:
# calculate the vector degrees of size (n) which contains d_j
degrees = A.sum(axis=0)

In [ ]:
# the degree of node 'Woman1' and 'Javert'
# here, using the double == gives a boolean mask, which subsequently only selects 
# entries that are true
neigh_wmn = degrees[names=='Woman1']
neigh_javert =  degrees[names=='Javert']
f'Woman1 has {neigh_wmn} pair appearences, Javert has {neigh_javert} pair appearences'

- Now we want to compute the matrix $S$. For this, we use [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html).

In [ ]:
# compute the matrix S
# s_ij = a_ij / d_j
# TO FILL

- Propose a piece of code to check that your $S$ matrix makes sense.

In [ ]:
# TO FILL

- Compute G from previous ingredients.

In [ ]:
# compute G
# TO FILL

## 1.c Running the power iteration

We now have all the ingredients to run PageRank. We initialize the weight vector $w$ uniformly and then update it iteratively via $w = Gw$ until the changes per iteration are not too big (the iterations converge).

We measure this change at every iteration, track it and plot the convergence.

- Initilize $w$ uniformly with $1/n$.

In [ ]:
# create a vector w of length n with weight uniformly distributed
# TO FILL

- Run the pagerank iteratively until w changes less than 1e-10 between two iterations. Keep track of the distance between the updated and old $w$ in a list, so that we can visualize it after.

In [ ]:
# track distance and convergence
dist = 1e300
dists = []

while dist > 1e-10:
# TO FILL

- Plot the list of distances using `plt.plot` and label the x (time) and y (distance) axis accordingly.

In [ ]:
# we plot the list of the distance values as a function of time
plt.axhline(np.linalg.norm(w - G @ w),color='red',linestyle='dotted')
# TO FILL

plt.xlabel(
plt.ylabel(
plt.title(

## 1.d Comparison against the `np.linalg` eigendecomposition 

For this we use numpy `linalg` module to compute the eigenvalues and vectors for a matrix. Since the poweriteration is computing $w$, such that $w=Gw$, we need to check for the eigenvalue $\lambda=1$.

However, $G$ may not be symmetric, so some eigenvalues may be complex. Therefore, the eigenvalue $1$ we are be looking for is the largest in complex norm.

In [ ]:
# use numpys code to do the eigenvalue decomp
eigenvalues, eigenvectors = np.linalg.eig(G)

In [ ]:
# the complex values eigenvalue and eigenvector at position 3
# the eigenvectors are organized in a matrix, where the ith column belongs to the ith eigenvalue
eigenvalues[3], eigenvectors[:,3]

In [ ]:
# We sort the eigenvalues by their norm, using `np.absolute`
sorted_idx = np.argsort(np.absolute(eigenvalues))

- Check what `np.argsort` does, and find the index from `sorted_idx` of the largest element in `eigenvalues`. Then, using this index, select the corresponding eigenvector and norm it to be a probability distribution. Then compare it to the pagerank values $w$. Do they match?

In [ ]:
# Take the index of the largest element, now positioned a the end of the array, due to ascending order.
idx = ... # TO FILL

In [ ]:
# select the respective eigenvector from the matrix
eigv = ... # TO FILL

In [ ]:
# norm the eigenvector to be a probability distribution, so that we can directly compare to pagerank
eigv = ... # TO FILL

In [ ]:
# compare the eigenv from the eigendecomposition with the the pagerank result
# TO FILL

## 1.e Interpreting the results

We are looking for central characters whose interactions are central. Are these the characters that also are the main characters in the story, such as percieved by respected scholars such as ChatGPT? 

> Can you give me a list of the ten main characters in Les Miserables, in the form of a python list?



```
main_characters = [
    "Valjean",
    "Javert",
    "Cosette",
    "Marius",
    "MmeThenardier",
    "Thenardier",
    "Fantine",
    "Gavroche",
    "Enjolras",
    "Myriel"
]
```

- To compare to those we need to extract the names of the nodes with the highest 10 page rank weights. (use `np.argsort` as in the ordering of the eigenvalues.

In [ ]:
# return a sorted index by page rank weight (ascending order)
sorted_importance = ... # TO FILL

In [ ]:
# the ten most central characters are those, with the highest page rank
ten_most_central_characters = ... # TO FILL
list(ten_most_central_characters)

... the central characters seem also be the main ones, they are interacting with one another a lot!

We can also look at them in the graph view:

In [ ]:
plot_les_miserables(highlight_names=ten_most_central_characters,layout='spring')

- It looks like the nodes with a high page rank value have also high degree. Check this by plotting the weight $w$ and the degree $d$ against one another on a scatterplot using `plt.scatter`. Don't forget to label the axes.

In [ ]:
# also, we plot the distribution of the page rank values w
x = np.linspace(0,w.max(),30)
y = np.linspace(0,degrees.max(),30)
plt.plot(x,y,c='grey',linestyle='dashed')
# TO FILL

# Exercise 2: The simple wikipedia link database

We now want to use PageRank on the [DBpedia link dataset](https://databus.dbpedia.org/dbpedia/generic/wikilinks/). We use the collection of all links between wikipedia pages that appear in the article collection of [simple english wikipedia](https://simple.wikipedia.org/wiki/Main_Page), which aims to explain things in understandable easy terms. It has ~ 1.3 Mio articles and categories, which is big - but not too big to run on a Laptop.

This will help us understand what the most central (or general) articiles in this wikipedia database are.


In [ ]:
from utils import load_simple_wikipedia
# load the dbpedia adjacency matrix and the names of the articles
names, A = load_simple_wikipedia()

- How many entities does the simple english wikipedia have exactly?

In [ ]:
n = ... # TO FILL
n

## 2.a Data Inspection: Does the loaded data 'make sense'?

Page # 13997 is [Electricity](https://simple.wikipedia.org/w/index.php?title=Electricity&oldid=8522069). We can use the list of names to verify this.

In [ ]:
names[13997]

The format of the array is now in [sparse row format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.html#scipy.sparse.csr_array), because this makes saving and computing matrix products much cheaper. You can use `type` to check the type of a variable (it is similar to a C++ class).

In [ ]:
type(A)

Let's look at the pages 'Electrictiy' is linking to.

The interface of the sparse matrix is slightly different than `np.array`. When we select a column, we first need to convert it back to a numpy array to use it like before.

In [ ]:
type(A[:,13997].toarray())

But then it has one extra dimension.

In [ ]:
A[:,13997].toarray().shape

So we squish it.

In [ ]:
A[:,13997].toarray().flatten().shape

And then we can use it to find the names of the pages that are linked from physics (are not 0).

In [ ]:
names[A[:,13997].toarray().flatten() != 0]

- Can we similarly find the pages linking to 'Electricity'? How do the list of the terms linking to and from the main page differ?

In [ ]:
# TO FILL

Some extra questions:

- How many entities have no outgoing links?

In [ ]:
# TO FILL

- How many entities have no incoming links?

In [ ]:
# TO FILL

## 2.b Implementing Page Rank using sparse matrix multiplication

To compute anything for wikipedia, we need to use the sparse matrix format and reexpress our PageRank matrix $G$ in an efficient way. Then you can easily process the data on a laptop. Recall that a sparse matrix has only few nonzero values. 
- We can check the percentage of nonzerovalues of A to see that this is indeed just a small fraction of the whole matrix.

In [ ]:
# TO FILL

Recall that in exercise 1, we computed the matrix $G$ as follows
$$
G = (1-\epsilon)S+\epsilon \frac{1}{n}
$$
$$
S_{ij} = A_{ij}/d_j\quad \mathrm{if } \,\,\,d_j > 0\ ,\quad
S_{ij} = 1/n\quad \mathrm{if }\,\,\,d_j = 0
$$
where $A$ is the adjacency matrix of the underlying graph, so $A_{ij}=1$ if there is an edge connection (link) pointing to $i$ from $j$ and it is zero if there is no link.
Then we run the following iteration until convergence
$$
w_{t+1} = G w_{t}
$$

Now, since we now have ~1,000,000 nodes and a dense matrix would have 1,000,000,000,000 entries, it is not very efficient to use a dense matrix (np.array) directly. This is why we went for the CSR format for $A$ previously (and it was already in that format when we imported it because saving it is also more efficient). 

However, in the pagerank update the matrices $S$ and $G$ are not as sparse, so it does not make sense to represent them as a sparse matrix. 

With a bit of thinking though, we realize that we can reformulate the computation of G at every iteration such that we only work with sparse matrices multiplcations and vector products directly.

We can still rescale $\tilde{A}=A_{ij}/d_j$ which maintains the zeros of A.


For the iterative update $w=Gw$ instead do the following calculation, where adding scalars is adding it to every entry of the vector
$$
w_{t+1} = (1-\epsilon) \left(\tilde{A}w_{t} + 1/n \sum_{j | d_j = 0} w_j \right) + \epsilon * 1/n
$$
Let's construct these things step by step.

In [ ]:
# First import the datatype, so that we can create a new diagonal matrix
# which divides the column j by the outgoing degree d_j
from scipy.sparse import csr_matrix

# calculate the row sums
row_sums = np.array(A.sum(axis=0)).flatten()

# prevent divide by zero error
row_sums[row_sums == 0.0] = 1.0

# create a diagonal matrix with the reciprocals of the row sums
division_matrix = csr_matrix((1 / row_sums, (range(len(row_sums)), range(len(row_sums)))), shape=(len(row_sums), len(row_sums)))

# apply the matrix 
A_tilde = A @ division_matrix

- Is the 'Electricity' column in `A_tilde` properly scaled to 1?

In [ ]:
# TO FILL

We want to build a binary mask of the $j | d_j = 0$, so we can use it in the algorithm.

In [ ]:
zero_row_mask = np.array(A_tilde.sum(axis=1)).flatten() == 0

And now this is an implementation of the sparsity adapted computation.

In [ ]:
# create a vector w of length n with weight uniformly distributed
w = np.ones(n)/n

# track distance and convergence
dist = np.inf
dists = []
# a while loop is run, until the condition at the beginning is False
while dist > 1e-7:
    w_new = (1-eps) * (A_tilde@w + w[zero_row_mask].sum()*1/n) + eps * 1/n
    dist = np.linalg.norm(w-w_new)
    w = w_new
    dists.append(dist)
    
#... should be done in a few seconds.

## 2.c Interpreting the results
- What are the ten most central articles? Use `np.argpartition` instead of `np.argsort`. Why is this the better choice? - check the documentation of both functions!

In [ ]:
ten_most_central_articles = # TO FILL
list(ten_most_central_articles)

## 2.d Bounus Question

As an open bonus exercise, we want to know which of the terms 'Physics', 'Chemistry', 'Computer Science', 'Biology', 'Mathematics' is most central in simple english wikipedia. Can you extract the pagerank of these articles and compare their ranks? 

# Exercice 3: Theoretical questions for the exam

1. Donnez la matrice d’adjacence $A \in \{0, 1\}^{5\times 5}$ du réseau de sites web donné ci-dessous. $A_{ij} = 1$ si le site web $j$
a un lien vers le site web $i$, sinon $A_{ij} = 0$. Les indices $i$ et $j$ indiquent respectivement les lignes et les colonnes de $A$.

![Image](data/graphe.png)

2. Quel site web a le plus grand degré sortant (out-degree) ? Quel site web a le plus grand degré entrant
(in-degree) ?

3. À quelle condition une matrice est-elle stochastique par colonnes (column-stochastic) ?